# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_df = pd.read_csv("weather_city_data.csv")
city_df.head()

,City_ID,City,Lat,Lng,Max_temp,Humidity,Clouds,Wind,Country,Date
0,0,kutum,14.20,24.67,91.76,18,97,8.72,SD,1593359456
1,1,alyangula,-13.85,136.42,66.20,93,87,9.91,AU,1593359456
2,2,fortuna,40.60,-124.16,57.00,87,90,12.75,US,1593359456
3,3,ushuaia,-54.80,-68.30,35.60,69,75,19.46,AR,1593359278
4,4,manuk mangkaw,4.80,119.85,80.13,80,100,5.03,PH,1593359456


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
gmaps.configure(api_key=g_key)


In [14]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
ncity_df = city_df.loc[(city_df["Max_temp"] < 80) & (city_df["Max_temp"] > 70) \
                                    & (city_df["Wind"] < 10) \
                                    & (city_df["Clouds"] == 0)].dropna()
ncity_df

,City_ID,City,Lat,Lng,Max_temp,Humidity,Clouds,Wind,Country,Date
143,143,jinchang,38.50,102.17,75.00,21,0,3.06,CN,1593359472
151,151,luau,-10.71,22.22,77.79,29,0,1.30,AO,1593359473
170,170,guerrero negro,27.98,-114.06,71.29,65,0,5.39,MX,1593359475
245,245,chapleau,47.84,-83.40,75.60,46,0,7.25,CA,1593359484
278,278,sinazongwe,-17.26,27.46,72.57,45,0,0.43,ZM,1593359487
303,303,longonjo,-12.91,15.25,78.80,26,0,5.82,AO,1593359490
358,358,labuhan,-6.88,112.21,77.86,83,0,7.45,ID,1593359496
373,373,prado,-17.34,-39.22,79.70,62,0,6.58,BR,1593359498
394,394,namibe,-15.20,12.15,72.66,58,0,9.46,AO,1593359500
399,399,pop,40.87,71.11,75.20,22,0,6.93,UZ,1593359500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df = ncity_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel_Name"] = ""
hotel_df


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel_Name
143,jinchang,CN,38.50,102.17,Mingshihui Jinchang International Hotel
151,luau,AO,-10.71,22.22,Kimbo Hotel
170,guerrero negro,MX,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
245,chapleau,CA,47.84,-83.40,Nemegosenda Lake Lodge
278,sinazongwe,ZM,-17.26,27.46,Lakeview Lodge
303,longonjo,AO,-12.91,15.25,
358,labuhan,ID,-6.88,112.21,bador sepret
373,prado,BR,-17.34,-39.22,INN CORNER RIVER - PRADO - BAHIA - HOUSING AND...
394,namibe,AO,-15.20,12.15,Hotel Chik Chik Namibe
399,pop,UZ,40.87,71.11,Ramz Hotel Pap


In [34]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))